In [1]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import ShuffleSplit
import lightgbm as lgb
import numpy as np
import pandas as pd

import os
import sys
sys.path.append('C:\\Users\\yu886\\OneDrive\\デスクトップ\\github\\Sony\\src')

from Processing.load_dataset import Load_dataset

from Engineering.dummy import engin
from Valid.validation import rmse

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

import warnings
warnings.simplefilter('ignore', category=UserWarning)
import matplotlib.pyplot as plt
from Processing.processing import Submission

In [4]:
train, test = Load_dataset()
df_train = engin(train)
df_test = engin(test)

target = df_train["pm25_mid"]

df_train = df_train.drop(["id","pm25_mid"], axis=1)
df_test = df_test.drop(["id"], axis=1)

FOLD = 5
NUM_ROUND = 10000
VERBOSE_EVAL = -1

params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'verbose': -1,

}

valid_scores = []
models = []
ss = ShuffleSplit(n_splits=FOLD, test_size=0.2, random_state=123)

for fold, (train_indices, valid_indices) in enumerate(ss.split(df_train)):
    X_train, X_valid = df_train.iloc[train_indices], df_train.iloc[valid_indices]
    y_train, y_valid = target.iloc[train_indices], target.iloc[valid_indices]
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid)

    model = lgb.train(
        params,
        lgb_train,
        valid_sets=lgb_eval,
        num_boost_round=NUM_ROUND,
        verbose_eval=VERBOSE_EVAL
    )

    y_valid_pred = model.predict(X_valid)
    MSE = mean_squared_error(y_valid, y_valid_pred)
    r2 = r2_score(y_valid, y_valid_pred)
    MAE = mean_absolute_error(y_valid, y_valid_pred)
    
    print(f'fold {fold} MSE: {MSE} RMSE: {np.sqrt(MSE)} R^2: {r2} MAE: {MAE}')
    valid_scores.append(np.sqrt(MSE))
    models.append(model)

cv_score = np.mean(valid_scores)
print(f'CV score: {cv_score}')

fold 0 MSE: 382.87909091762987 RMSE: 19.567296464193255 R^2: 0.7421696605305064 MAE: 14.01279212263285
fold 1 MSE: 378.4398750166998 RMSE: 19.45353117088771 R^2: 0.7431966188557351 MAE: 13.975672583613953
fold 2 MSE: 378.5033669780846 RMSE: 19.455162990272907 R^2: 0.7450432187964875 MAE: 13.91412743034694
fold 3 MSE: 376.9238875071373 RMSE: 19.414527743603173 R^2: 0.745175663285076 MAE: 13.901213346827976
fold 4 MSE: 380.6614723432478 RMSE: 19.510547720226814 R^2: 0.7425094865811841 MAE: 14.005420651210784
CV score: 19.48021321783677


In [2]:
def add(df_train):
    df_train["co_avg"] = df_train["co_mid"] / df_train["co_cnt"]
    df_train["o3_avg"] = df_train["o3_mid"] / df_train["o3_cnt"]
    df_train["so2_avg"] = df_train["so2_mid"] / df_train["so2_cnt"]
    df_train["no2_avg"] = df_train["no2_mid"] / df_train["no2_cnt"]
    df_train["temperature_avg"] = df_train["temperature_mid"] / df_train["temperature_cnt"]
    df_train["humidity_avg"] = df_train["humidity_mid"] / df_train["humidity_cnt"]
    df_train["pressure_avg"] = df_train["pressure_mid"] / df_train["pressure_cnt"]
    df_train["ws_avg"] = df_train["ws_mid"] / df_train["ws_cnt"]
    df_train["dew_avg"] = df_train["dew_mid"] / df_train["dew_cnt"]
    return df_train

In [10]:
from Engineering.dummy import engin_2
train, test = Load_dataset()

In [11]:
df_train = engin_2(train)
df_test = engin_2(test)

target = df_train["pm25_mid"]

df_train = df_train.drop(["id","pm25_mid"], axis=1)
df_test = df_test.drop(["id"], axis=1)

df_train = add(df_train)
df_test = add(df_test)

In [13]:
FOLD = 5
NUM_ROUND = 10000
VERBOSE_EVAL = -1

params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'verbose': -1,
    'learning_rate':0.01
}

valid_scores = []
models = []
ss = ShuffleSplit(n_splits=FOLD, test_size=0.25, random_state=123)

for fold, (train_indices, valid_indices) in enumerate(ss.split(df_train)):
    X_train, X_valid = df_train.iloc[train_indices], df_train.iloc[valid_indices]
    y_train, y_valid = target.iloc[train_indices], target.iloc[valid_indices]
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid)

    model = lgb.train(
        params,
        lgb_train,
        valid_sets=lgb_eval,
        num_boost_round=NUM_ROUND,
        verbose_eval=VERBOSE_EVAL
    )

    y_valid_pred = model.predict(X_valid)
    MSE = mean_squared_error(y_valid, y_valid_pred)
    r2 = r2_score(y_valid, y_valid_pred)
    MAE = mean_absolute_error(y_valid, y_valid_pred)
    
    print(f'fold {fold} MSE: {MSE} RMSE: {np.sqrt(MSE)} R^2: {r2} MAE: {MAE}')
    valid_scores.append(np.sqrt(MSE))
    models.append(model)

cv_score = np.mean(valid_scores)
print(f'CV score: {cv_score}')

fold 0 MSE: 378.92969221035787 RMSE: 19.466116515893916 R^2: 0.7447781651490184 MAE: 14.02358476095657
fold 1 MSE: 381.49113483354824 RMSE: 19.53179804404982 R^2: 0.7430885824994896 MAE: 14.060204966041448
fold 2 MSE: 375.4824349022953 RMSE: 19.37736914295373 R^2: 0.7457489490053983 MAE: 13.928531210536878
fold 3 MSE: 376.9369138454015 RMSE: 19.414863219847867 R^2: 0.7449231981004755 MAE: 14.000450148463386
fold 4 MSE: 385.5805330286938 RMSE: 19.636204649287343 R^2: 0.7408021152385927 MAE: 14.10605016084066
CV score: 19.485270314406534


In [19]:
pred_1 = models[0].predict(df_train)
pred_2 = models[1].predict(df_train)
pred_3 = models[2].predict(df_train)
pred_4 = models[3].predict(df_train)
pred_5 = models[4].predict(df_train)

pred = pd.DataFrame(np.stack([pred_1, pred_2, pred_3 ,pred_4, pred_5], axis=1))

## 4/21

In [3]:
from Engineering.dummy import engin_2
train, test = Load_dataset()

In [4]:
def add_2(df):
    df["discomfort"] = 0.81*df["temperature_mid"] + 0.01*df["humidity_mid"]*(0.99*df["temperature_mid"] - 14.3) + 46.3
    return df

In [20]:
df_train = engin_2(train)
df_test = engin_2(test)

target = df_train["pm25_mid"]

df_train = df_train.drop(["id","pm25_mid"], axis=1)
df_test = df_test.drop(["id"], axis=1)

# df_train = add(df_train)
# df_test = add(df_test)
df_train = add_2(df_train)

FOLD = 5
NUM_ROUND = 10000
VERBOSE_EVAL = -1

params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'verbose': -1,
    'learning_rate':0.01
}

valid_scores = []
models = []
ss = ShuffleSplit(n_splits=FOLD, test_size=0.25, random_state=123)

for fold, (train_indices, valid_indices) in enumerate(ss.split(df_train)):
    X_train, X_valid = df_train.iloc[train_indices], df_train.iloc[valid_indices]
    y_train, y_valid = target.iloc[train_indices], target.iloc[valid_indices]
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid)

    model = lgb.train(
        params,
        lgb_train,
        valid_sets=lgb_eval,
        num_boost_round=NUM_ROUND,
        verbose_eval=VERBOSE_EVAL
    )

    y_valid_pred = model.predict(X_valid)
    MSE = mean_squared_error(y_valid, y_valid_pred)
    r2 = r2_score(y_valid, y_valid_pred)
    MAE = mean_absolute_error(y_valid, y_valid_pred)
    
    print(f'fold {fold} MSE: {MSE} RMSE: {np.sqrt(MSE)} R^2: {r2} MAE: {MAE}')
    valid_scores.append(np.sqrt(MSE))
    models.append(model)

cv_score = np.mean(valid_scores)
print(f'CV score: {cv_score}')

fold 0 MSE: 379.1868856530105 RMSE: 19.472721577966716 R^2: 0.744604936754161 MAE: 14.053711346482668
fold 1 MSE: 381.9228595119203 RMSE: 19.54284676069278 R^2: 0.7427978418007866 MAE: 14.070579694264332
fold 2 MSE: 376.99452150747607 RMSE: 19.41634676007503 R^2: 0.744725067265998 MAE: 13.955855807821015
fold 3 MSE: 376.90201868734874 RMSE: 19.41396452781731 R^2: 0.744946811986489 MAE: 14.002243000701066
fold 4 MSE: 384.1953896263069 RMSE: 19.60090277579854 R^2: 0.7417332468939432 MAE: 14.082433793796241
CV score: 19.489356480470075


In [5]:
def add_2(df):
    df["discomfort"] = 0.81*df["temperature_mid"] + 0.01*df["humidity_mid"]*(0.99*df["temperature_mid"] - 14.3) + 46.3
    return df

def add_3(df):
#     df["sum_count"] = (df_train["co_cnt"] + df_train["o3_cnt"] + df_train["so2_cnt"] + df_train["no2_cnt"] +
#                        df_train["temperature_cnt"] + df_train["humidity_cnt"] + df_train["pressure_cnt"] + df_train["ws_cnt"] + df_train["dew_cnt"])
    df["avg_count"] = (df_train["co_cnt"] + df_train["o3_cnt"] + df_train["so2_cnt"] + df_train["no2_cnt"] +
                       df_train["temperature_cnt"] + df_train["humidity_cnt"] + df_train["pressure_cnt"] + df_train["ws_cnt"] + df_train["dew_cnt"]) / 9
    return df

In [43]:
df_train = engin_2(train)
df_test = engin_2(test)

target = df_train["pm25_mid"]

df_train = df_train.drop(["id","pm25_mid"], axis=1)
df_test = df_test.drop(["id"], axis=1)

df_train = add(df_train)
df_test = add(df_test)

df_train = add_2(df_train)
df_test = add_2(df_test)

df_train = add_3(df_train)
df_test = add_3(df_test)

FOLD = 5
NUM_ROUND = 10000
VERBOSE_EVAL = -1

params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'verbose': -1,
    'learning_rate':0.01
}

valid_scores = []
models = []
ss = ShuffleSplit(n_splits=FOLD, test_size=0.25, random_state=123)

for fold, (train_indices, valid_indices) in enumerate(ss.split(df_train)):
    X_train, X_valid = df_train.iloc[train_indices], df_train.iloc[valid_indices]
    y_train, y_valid = target.iloc[train_indices], target.iloc[valid_indices]
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid)

    model = lgb.train(
        params,
        lgb_train,
        valid_sets=lgb_eval,
        num_boost_round=NUM_ROUND,
        verbose_eval=VERBOSE_EVAL
    )

    y_valid_pred = model.predict(X_valid)
    MSE = mean_squared_error(y_valid, y_valid_pred)
    r2 = r2_score(y_valid, y_valid_pred)
    MAE = mean_absolute_error(y_valid, y_valid_pred)
    
    print(f'fold {fold} MSE: {MSE} RMSE: {np.sqrt(MSE)} R^2: {r2} MAE: {MAE}')
    valid_scores.append(np.sqrt(MSE))
    models.append(model)

cv_score = np.mean(valid_scores)
print(f'CV score: {cv_score}')

pred_1 = models[0].predict(df_test)
pred_2 = models[1].predict(df_test)
pred_3 = models[2].predict(df_test)
pred_4 = models[3].predict(df_test)
pred_5 = models[4].predict(df_test)

pred = np.stack([pred_1, pred_2, pred_3 ,pred_4, pred_5], axis=1)
pred = np.mean(pred, axis=1)

index = test["id"]
Submission(index, pred, name='light_count_avg')

fold 0 MSE: 379.12483348274657 RMSE: 19.471128202616985 R^2: 0.7446467309684355 MAE: 14.030395577363052
fold 1 MSE: 382.25134205460444 RMSE: 19.55124911750153 R^2: 0.7425766284934263 MAE: 14.067602134387478
fold 2 MSE: 375.3423122190416 RMSE: 19.373753178438132 R^2: 0.7458438305129573 MAE: 13.929319115485793
fold 3 MSE: 375.7520555190647 RMSE: 19.384324995187857 R^2: 0.7457250030218951 MAE: 13.9928326081164
fold 4 MSE: 384.74002409783736 RMSE: 19.61479095218293 R^2: 0.741367128558352 MAE: 14.097294480595053
CV score: 19.479049289185486


In [38]:
importance = pd.DataFrame(sorted(models[1].feature_importance(), reverse=True), index=df_train.columns, columns=['importance'])
importance[-5:]
# plt.figure(figsize=(15,6))
# plt.bar(df_train.columns[:20], np.ravel(importance.values)[:20])

,importance
pressure_avg,102
ws_avg,95
dew_avg,71
discomfort,21
avg_count,0


In [11]:
df_train = engin_2(train)
df_test = engin_2(test)

target = df_train["pm25_mid"]

df_train = df_train.drop(["id","pm25_mid"], axis=1)
df_test = df_test.drop(["id"], axis=1)

df_train = add(df_train)
df_test = add(df_test)

df_train = add_2(df_train)
df_test = add_2(df_test)

df_train = add_3(df_train)
df_test = add_3(df_test)

# df_train = pd.concat([df_train, rains], axis=1)
df_train.columns

Index(['co_cnt', 'co_min', 'co_mid', 'co_max', 'o3_cnt', 'o3_min', 'o3_mid',
       'o3_max', 'so2_cnt', 'so2_min', 'so2_mid', 'so2_max', 'no2_cnt',
       'no2_min', 'no2_mid', 'no2_max', 'temperature_cnt', 'temperature_min',
       'temperature_mid', 'temperature_max', 'humidity_cnt', 'humidity_min',
       'humidity_mid', 'humidity_max', 'pressure_cnt', 'pressure_min',
       'pressure_mid', 'pressure_max', 'ws_cnt', 'ws_min', 'ws_mid', 'ws_max',
       'dew_cnt', 'dew_min', 'dew_mid', 'dew_max', 'Australia', 'Belgium',
       'Bosnia and Herzegovina', 'Brazil', 'Canada', 'Chile', 'China',
       'Croatia', 'France', 'Germany', 'Hungary', 'India', 'Iran', 'Israel',
       'Italy', 'Japan', 'Mexico', 'Netherlands', 'Poland', 'Russia', 'Serbia',
       'South Africa', 'South Korea', 'Spain', 'Taiwan', 'Thailand', 'Turkey',
       'United Kingdom', 'United States', 'Vietnam', 'co_std', 'o3_std',
       'so2_std', 'no2_std', 'temperature_std', 'humidity_std', 'pressure_std',
       'ws_

In [12]:
FOLD = 5
NUM_ROUND = 10000
VERBOSE_EVAL = -1

params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'verbose': -1,
    
    'learning_rate': 0.019544378433864855,
     'max_depth': 9,
#      'early_stopping_rounds': 427,
     'n_estimators': 7000,
     'lambda_l1': 9.65363914639908,
     'lambda_l2': 0.039176091009943646,
     'num_leaves': 101,
     'feature_fraction': 0.5248775546494105,
     'bagging_fraction': 0.9795877647513113,
     'bagging_freq': 6,
     'min_child_samples': 55,
}

valid_scores = []
models = []
ss = ShuffleSplit(n_splits=FOLD, test_size=0.25, random_state=123)

for fold, (train_indices, valid_indices) in enumerate(ss.split(df_train)):
    X_train, X_valid = df_train.iloc[train_indices], df_train.iloc[valid_indices]
    y_train, y_valid = target.iloc[train_indices], target.iloc[valid_indices]
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid)

    model = lgb.train(
        params,
        lgb_train,
        valid_sets=lgb_eval,
        num_boost_round=NUM_ROUND,
        verbose_eval=VERBOSE_EVAL
    )

    y_valid_pred = model.predict(X_valid)
    MSE = mean_squared_error(y_valid, y_valid_pred)
    r2 = r2_score(y_valid, y_valid_pred)
    MAE = mean_absolute_error(y_valid, y_valid_pred)
    
    print(f'fold {fold} MSE: {MSE} RMSE: {np.sqrt(MSE)} R^2: {r2} MAE: {MAE}')
    valid_scores.append(np.sqrt(MSE))
    models.append(model)

cv_score = np.mean(valid_scores)
print(f'CV score: {cv_score}')

fold 0 MSE: 372.961277663784 RMSE: 19.31220540652424 R^2: 0.7487980921778095 MAE: 13.841745426128185
fold 1 MSE: 375.47607027085684 RMSE: 19.377204913786116 R^2: 0.7471393680146823 MAE: 13.863789009965123
fold 2 MSE: 367.71301483121505 RMSE: 19.175844566308285 R^2: 0.751009869450864 MAE: 13.724184506537302
fold 3 MSE: 371.1573098339596 RMSE: 19.265443411298882 R^2: 0.7488343112160483 MAE: 13.822457680275175
fold 4 MSE: 378.4815890392726 RMSE: 19.454603286607327 R^2: 0.7455742214744663 MAE: 13.890083740610734
CV score: 19.317060316904968


In [13]:
pred_1 = models[0].predict(df_test)
pred_2 = models[1].predict(df_test)
pred_3 = models[2].predict(df_test)
pred_4 = models[3].predict(df_test)
pred_5 = models[4].predict(df_test)

pred = np.stack([pred_1, pred_2, pred_3 ,pred_4, pred_5], axis=1)
pred = np.mean(pred, axis=1)

index = test["id"]
Submission(index, pred, name='light_auto_count_avg_var')

In [3]:
from Engineering.dummy import engin_2

In [6]:
train, test = Load_dataset()

df_train = engin_2(train)
df_test = engin_2(test)

df_train = add(df_train)
df_test = add(df_test)

df_train = add_2(df_train)
df_test = add_2(df_test)

df_train = add_3(df_train)
df_test = add_3(df_test)

# df_train = pd.concat([df_train, rains], axis=1)
df_train.columns
df_test.columns

Index(['id', 'co_cnt', 'co_min', 'co_mid', 'co_max', 'o3_cnt', 'o3_min',
       'o3_mid', 'o3_max', 'so2_cnt', 'so2_min', 'so2_mid', 'so2_max',
       'no2_cnt', 'no2_min', 'no2_mid', 'no2_max', 'temperature_cnt',
       'temperature_min', 'temperature_mid', 'temperature_max', 'humidity_cnt',
       'humidity_min', 'humidity_mid', 'humidity_max', 'pressure_cnt',
       'pressure_min', 'pressure_mid', 'pressure_max', 'ws_cnt', 'ws_min',
       'ws_mid', 'ws_max', 'dew_cnt', 'dew_min', 'dew_mid', 'dew_max',
       'Australia', 'Belgium', 'Bosnia and Herzegovina', 'Brazil', 'Canada',
       'Chile', 'China', 'Croatia', 'France', 'Germany', 'Hungary', 'India',
       'Iran', 'Israel', 'Italy', 'Japan', 'Mexico', 'Netherlands', 'Poland',
       'Russia', 'Serbia', 'South Africa', 'South Korea', 'Spain', 'Taiwan',
       'Thailand', 'Turkey', 'United Kingdom', 'United States', 'Vietnam',
       'co_std', 'o3_std', 'so2_std', 'no2_std', 'temperature_std',
       'humidity_std', 'pressure_std'

In [7]:
import re
df_train = df_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df_test = df_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [8]:
x = df_train.columns
for i in range(len(x)):
    print(x[i])

id
co_cnt
co_min
co_mid
co_max
o3_cnt
o3_min
o3_mid
o3_max
so2_cnt
so2_min
so2_mid
so2_max
no2_cnt
no2_min
no2_mid
no2_max
temperature_cnt
temperature_min
temperature_mid
temperature_max
humidity_cnt
humidity_min
humidity_mid
humidity_max
pressure_cnt
pressure_min
pressure_mid
pressure_max
ws_cnt
ws_min
ws_mid
ws_max
dew_cnt
dew_min
dew_mid
dew_max
pm25_mid
Australia
Belgium
BosniaandHerzegovina
Brazil
Canada
Chile
China
Croatia
France
Germany
Hungary
India
Iran
Israel
Italy
Japan
Mexico
Netherlands
Poland
Russia
Serbia
SouthAfrica
SouthKorea
Spain
Taiwan
Thailand
Turkey
UnitedKingdom
UnitedStates
Vietnam
co_std
o3_std
so2_std
no2_std
temperature_std
humidity_std
pressure_std
ws_std
dew_std
co_avg
o3_avg
so2_avg
no2_avg
temperature_avg
humidity_avg
pressure_avg
ws_avg
dew_avg
discomfort
avg_count


In [9]:
df_train.to_csv("C:/Users/yu886/OneDrive/デスクトップ/github/Sony/Process/train_01.csv")
df_test.to_csv("C:/Users/yu886/OneDrive/デスクトップ/github/Sony/Process/test_01.csv")